In [5]:
import torch
import metaspore as ms
import numpy as np
import yaml
import argparse
import sys 
from torch import nn
import operator 
from operator import itemgetter
import pyspark

sys.path.append('../../') 
# from python.widedeep import WideDeep
from metaspore.nn import WideAndDeepModule as WideDeep

In [6]:
!aws s3 cp  --recursive ./schema/widedeep/ s3://dmetasoul-bucket/demo/movielens/schema/widedeep/

upload: schema/widedeep/combine_column_schema to s3://dmetasoul-bucket/demo/movielens/schema/widedeep/combine_column_schema
upload: schema/widedeep/.ipynb_checkpoints/column_schema-checkpoint to s3://dmetasoul-bucket/demo/movielens/schema/widedeep/.ipynb_checkpoints/column_schema-checkpoint
upload: schema/widedeep/.ipynb_checkpoints/combine_column_schema-checkpoint to s3://dmetasoul-bucket/demo/movielens/schema/widedeep/.ipynb_checkpoints/combine_column_schema-checkpoint
upload: schema/widedeep/column_schema to s3://dmetasoul-bucket/demo/movielens/schema/widedeep/column_schema


In [7]:
!aws s3 ls s3://dmetasoul-bucket/demo/movielens/schema/widedeep/

                           PRE .ipynb_checkpoints/
2022-02-10 08:22:37        117 column_schema
2022-02-10 08:22:37         80 combine_column_schema


In [8]:
cp ../../python.zip .

In [9]:
def load_config(path):
    params=dict()
    with open(path,'r') as stream:
        params=yaml.load(stream,Loader=yaml.FullLoader)
        print('Debug--load config:',params)
    return params

In [10]:
def init_spark():
    spark_confs={
        "spark.network.timeout":"500",
        "spark.submit.pyFiles":"python.zip",
        # "spark.ui.showConsoleProgress": "false",
        #"spark.kubernetes.executor.deleteOnTermination":"false",
    }
    spark_session=ms.spark.get_session(local=local,
                                   app_name=app_name,
                                   batch_size=batch_size,
                                   worker_count=worker_count,
                                   server_count=server_count,
                                   worker_memory=worker_memory,
                                   server_memory=server_memory,
                                   coordinator_memory=coordinator_memory,
                                   spark_confs=spark_confs)
    
    sc = spark_session.sparkContext
    print('Debug -- spark init')
    print('Debug -- version:', sc.version)   
    print('Debug -- applicaitonId:', sc.applicationId)
    print('Debug -- uiWebUrl:', sc.uiWebUrl)
    return spark_session


def stop_spark(spark):
    print('Debug--spark stop')
    spark.sparkContext.stop()


def read_dataset(spark):
    train_dataset=spark.read.parquet(train_path)
    test_dataset=spark.read.parquet(test_path)

    print('Debug -- match train dataset sample:')
    train_dataset.show(10)
    print('Debug--match train test dataset sample:')
    test_dataset.show(10)
    
    print('Debug -- train dataset positive count:', train_dataset[train_dataset['label']=='1'].count())
    print('Debug -- train dataset negative count:', train_dataset[train_dataset['label']=='0'].count())
    print('Debug -- test dataset count:', test_dataset.count())

    return train_dataset,test_dataset
    
def train(spark, trian_dataset, **model_params):
    # module = WideDeep(column_name_path=column_name_path,
    #                combine_schema_path=combine_schema_path)
    module = WideDeep(wide_column_name_path=column_name_path,
                      wide_combine_schema_path=combine_schema_path,
                      deep_sparse_column_name_path=column_name_path,
                      deep_sparse_combine_schema_path=combine_schema_path,
                      deep_dense_hidden_units=[1024,512,256,128,1],
                      wide_embedding_size=embedding_size,
                      deep_sparse_embedding_size=embedding_size)

    estimator=ms.PyTorchEstimator(module=module,
                                  worker_count=worker_count,
                                  server_count=server_count,
                                  model_out_path=model_out_path,
                                  input_label_column_index=input_label_column_index,
                                  metric_update_interval=100)
    model = estimator.fit(trian_dataset)
    import subprocess
    run_res=subprocess.run(['aws', 's3', 'ls', model_out_path], 
                            stdout=subprocess.PIPE)
    print('Debug -- model output:')
    print("%s\n"%run_res.stdout.decode("utf-8"))
    return model

def transform(spark,model,test_dataset):
    test_result = model.transform(test_dataset)
    print('Debug -- test result sample:')
    test_result.show(20)
    return test_result

def evaluate(spark,test_result,test_user=100):
    pass

In [11]:
if __name__=="__main__":
    print('Debug -- Movielens Recall Demo')
    params=load_config('widedeep.yaml')
    locals().update(params)
    spark = init_spark()
    train_dataset, test_dataset = read_dataset(spark)
    model = train(spark, train_dataset)
    test_result = transform(spark, model, test_dataset)
    stop_spark(spark)

Debug -- Movielens Recall Demo
Debug--load config: {'app_name': 'WideDeep Demo', 'local': False, 'worker_count': 4, 'server_count': 2, 'batch_size': 60, 'worker_memory': '5G', 'server_memory': '5G', 'coordinator_memory': '5G', 'train_path': 's3://dmetasoul-bucket/demo/movielens/rank/train.parquet', 'test_path': 's3://dmetasoul-bucket/demo/movielens/rank/test.parquet', 'column_name_path': 's3://dmetasoul-bucket/demo/schema/item_column_schema', 'combine_schema_path': 's3://dmetasoul-bucket/demo/schema/item_combine_schema', 'model_in_path': None, 'model_out_path': 's3://dmetasoul-bucket/demo/movielens/model/widedeep/model_out/', 'model_export_path': None, 'model_version': None, 'experiment_name': None, 'input_label_column_index': 0, 'embedding_size': 16, 'net_dropout': 0.0, 'adam_learning_rate': 1e-05, 'ftrl_learning_rate': 0.02, 'ftrl_smothing_rate': 1.0, 'ftrl_l1_regularization': 1.0, 'ftrl_l2_regularization': 1.0, 'training_epoches': 1, 'shuffle_training_dataset': False}


22/02/10 08:22:43 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


Debug -- spark init
Debug -- version: 3.1.2
Debug -- applicaitonId: spark-application-1644481365471
Debug -- uiWebUrl: http://jupyter.my.nginx.test/hub/user-redirect/proxy/4040/jobs/


Debug -- match train dataset sample:


+-----+-------+------+---+----------+-----+--------+--------------------+--------------------+----------+--------------------+
|label|user_id|gender|age|occupation|  zip|movie_id|    recent_movie_ids|               genre|last_movie|          last_genre|
+-----+-------+------+---+----------+-----+--------+--------------------+--------------------+----------+--------------------+
|    1|   1261|     M| 18|         4|40205|     588|14594101383212...|AnimationChildre...|      2791|              Comedy|
|    0|   5098|     M| 50|         7|19380|    2287|12838993035120...| Sci-FiThrillerWar|      1287|ActionAdventure...|
|    1|   4771|     F| 25|         4|04101|     151|4917212801215...|   DramaRomanceWar|       613|       DramaRomance|
|    1|   2599|     M| 25|        12|53154|    3699|15276713175370...|AdventureDramaR...|      2011|       ComedySci-Fi|
|    1|   2887|     M| 25|         3|66205|    2150|3392482252708...|              Comedy|       252|      

...

2022-02-10 08:26:09.581 -- auc: 0.6368976516778828, Δauc: 0.756227458671667, pcoc: 0.4559938865570009, Δpcoc: 0.5370781861000182, #instance: 611841
2022-02-10 08:26:10.624 -- auc: 0.6375735488512995, Δauc: 0.756017847401601, pcoc: 0.45676191104175834, Δpcoc: 0.5342203194385159, #instance: 617841
2022-02-10 08:26:13.220 -- auc: 0.6381192127532345, Δauc: 0.7548956877997358, pcoc: 0.45761086880692614, Δpcoc: 0.5450248020003067, #instance: 623841
2022-02-10 08:26:14.371 -- auc: 0.6384763603452908, Δauc: 0.736493873034582, pcoc: 0.458481024514764, Δpcoc: 0.5495325728393634, #instance: 629841
2022-02-10 08:26:17.014 -- auc: 0.6388345162564777, Δauc: 0.7424875305853567, pcoc: 0.45932604045266406, Δpcoc: 0.5489977782938904, #instance: 635841
2022-02-10 08:26:18.192 -- auc: 0.6393678813289139, Δauc: 0.7471023378570834, pcoc: 0.46016215602176114, Δpcoc: 0.548163586401118, #instance: 641841
2022-02-10 08:26:20.765 -- auc: 0.6398417796577476, Δauc: 0.7487734861779717, pcoc: 0.4609707942882186, Δpc

2022-02-10 08:26:46.110 -- auc: 0.6452740548179063, Δauc: 1.0, pcoc: 0.47162257436053684, Δpcoc: nan, #instance: 725841
2022-02-10 08:26:46.119 -- auc: 0.6454607735784468, Δauc: 0.7544117303013289, pcoc: 0.4719435269172896, Δpcoc: 0.5737701857243944, #instance: 728102
2022-02-10 08:26:46.127 -- auc: 0.6456554989727139, Δauc: 0.748656947331454, pcoc: 0.4723596581574847, Δpcoc: 0.5691884096424372, #instance: 731230
2022-02-10 08:26:46.135 -- auc: 0.6459412415610493, Δauc: 0.7349312590236803, pcoc: 0.4724984203815541, Δpcoc: 0.5047797194100128, #instance: 734375


[2022-02-10 08:26:51.641] [info] C[0]:9 has stopped.
[2022-02-10 08:26:51.642] [info] PS job with coordinator address 10.0.1.114:35933 stopped.
Debug -- model output:
2022-02-10 08:26:46         64 _deep_dense.0.bias__dense_data.dat
2022-02-10 08:26:46        445 _deep_dense.0.bias__dense_meta.json
2022-02-10 08:26:47        128 _deep_dense.0.bias__dense_state.dat
2022-02-10 08:26:51         64 _deep_dense.0.running_mean__dense_data.dat
2022-02-10 08:26:51        373 _deep_dense.0.running_mean__dense_meta.json
2022-02-10 08:26:51         64 _deep_dense.0.running_var__dense_data.dat
2022-02-10 08:26:51        372 _deep_dense.0.running_var__dense_meta.json
2022-02-10 08:26:46         64 _deep_dense.0.weight__dense_data.dat
2022-02-10 08:26:46        447 _deep_dense.0.weight__dense_meta.json
2022-02-10 08:26:46        128 _deep_dense.0.weight__dense_state.dat
2022-02-10 08:26:47       4096 _deep_dense.1.bias__dense_data.dat
2022-02-10 08:26:47        453 _deep_dense.1.bias__dense_meta.jso

2022-02-10 08:27:14.749 -- auc: 0.7563887725178048, Δauc: 0.7563887725178048, pcoc: 0.5572457710227254, Δpcoc: 0.5572457710227254, #instance: 743
2022-02-10 08:27:14.757 -- auc: 0.7666383556172571, Δauc: 0.7718625872249061, pcoc: 0.5531430833979952, Δpcoc: 0.5510417938232421, #instance: 2217
2022-02-10 08:27:14.766 -- auc: 0.7569659695327484, Δauc: 0.7458632824198862, pcoc: 0.5639113548928859, Δpcoc: 0.5807361744163493, #instance: 3690
2022-02-10 08:27:14.774 -- auc: 0.759942509184566, Δauc: 0.7704392717497557, pcoc: 0.5673704797019375, Δpcoc: 0.5818359107216082, #instance: 4612
[2022-02-10 08:27:14.792] [info] C[0]:9 has stopped.
[2022-02-10 08:27:14.794] [info] PS job with coordinator address 10.0.1.114:41751 stopped.
Debug -- test result sample:
+-----+-------+------+---+----------+-----+--------+--------------------+--------------------+----------+--------------------+-------------------+
|label|user_id|gender|age|occupation|  zip|movie_id|    recent_movie_ids|               genre|

22/02/10 08:27:14 WARN ExecutorPodsWatchSnapshotSource: Kubernetes client has been closed (this is expected if the application is shutting down.)
